In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
data_dir = 'Dataset/GAN/BigGan/train'
# Define transforms for the training and validation sets
data_transforms ={
    "train_transforms": transforms.Compose([transforms.RandomRotation(30),
                                           transforms.RandomResizedCrop(299), 
                                           transforms.RandomHorizontalFlip(), 
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406], 
                                                                [0.229, 0.224, 0.225])]),
   "valid_transforms": transforms.Compose([transforms.Resize(300),
                                           transforms.CenterCrop(299),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])]), 
    "test_transforms": transforms.Compose([transforms.Resize(300),
                                           transforms.CenterCrop(299),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])
}
import numpy as np

# Set the split ratios for training, validation, and test data
train_data_ratio = 0.8
valid_data_ratio = 0.2

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(data_dir, transform=data_transforms["train_transforms"])
valid_data = datasets.ImageFolder(data_dir, transform=data_transforms["valid_transforms"])

# Obtain training indices that will be used for validation and test
num_train = len(train_data)
indices = list(range(num_train))

# Calculate the number of samples for training, validation, and test sets
train_count = int(train_data_ratio * num_train)
valid_count = int(valid_data_ratio * num_train)
test_count = num_train - train_count - valid_count

# Split the indices into training, validation, and test sets
train_idx = indices[:train_count]
valid_idx = indices[train_count:train_count + valid_count]
test_idx = indices[train_count + valid_count:]

# Randomly remove 10% of samples from the training set
remove_count = int(0.1 * train_count)
np.random.shuffle(train_idx)
train_idx = train_idx[:-remove_count]

# Print the lengths of training, validation, and test sets
print(len(train_idx), len(valid_idx), len(test_idx))

# Print the percentage of total data each set represents
print("Training", len(train_idx), np.sum(len(train_idx) / num_train))
print("Validation", valid_count, np.sum(len(valid_idx) / num_train))
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
# trainloader = torch.utils.data.DataLoader(train_data, batch_size = 64, shuffle = True,num_workers=100)
trainloader = torch.utils.data.DataLoader(train_data, batch_size = 16, shuffle = True,num_workers=100)
validloader = torch.utils.data.DataLoader(valid_data, batch_size = 32, sampler = valid_sampler)
classes=['ai', 'nature']
model_transfer = models.resnet50(pretrained=False)

# Check if GPU is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_transfer = model_transfer.cuda()
for param in model_transfer.parameters():
    param.requires_grad=True
n_inputs = model_transfer.fc.in_features #refer to the fully connected layer only

# Add last linear layer (n_inputs -> 4 classes). In this case the ouput is 4 classes
# New layer automatically has requires_grad = True
last_layer = nn.Linear(n_inputs, len(classes))

model_transfer.fc = last_layer

# If GPU is available, move the model to GPU
if use_cuda:
    model_transfer = model_transfer.cuda()
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.parameters(), lr=0.001, momentum=0.9)

538557 149599 0
Training 538557 0.7200008021443993
Validation 149599 0.2


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
import torch
from torchvision import models, transforms
from PIL import Image
from torch import nn, optim

classes = ['ai','nature']

model1 = models.resnet50(pretrained=True)
n_inputs = model1.fc.in_features 
last_layer = nn.Linear(n_inputs, len(classes))
model1.fc = last_layer
model1.load_state_dict(torch.load('Model/VQDM_Resnet50.pt'))

model2 = models.resnet50(pretrained=True)
n_inputs = model2.fc.in_features 
last_layer = nn.Linear(n_inputs, len(classes))
model2.fc = last_layer
model2.load_state_dict(torch.load('Model/GLIDE_Resnet50.pt'))

model3 = models.resnet50(pretrained=True)
n_inputs = model3.fc.in_features 
last_layer = nn.Linear(n_inputs, len(classes))
model3.fc = last_layer
model3.load_state_dict(torch.load('Model/GAN_Resnet50.pt'))

model4 = models.resnet50(pretrained=True)
n_inputs = model3.fc.in_features 
last_layer = nn.Linear(n_inputs, len(classes))
model4.fc = last_layer
model4.load_state_dict(torch.load('Model/SDM_Resnet50.pt'))

model5 = models.resnet50(pretrained=True)
n_inputs = model5.fc.in_features 
last_layer = nn.Linear(n_inputs, len(classes))
model5.fc = last_layer
model5.load_state_dict(torch.load('Midjourney_Resnet50.pt'))

modelX = models.resnet50(pretrained=True)
n_inputs = model3.fc.in_features 
last_layer = nn.Linear(n_inputs, len(classes))
modelX.fc = last_layer
modelX.load_state_dict(torch.load('Model/FineTuning40%Only.pt'))

# modelZ = models.resnet50()
# n_inputs = modelZ.fc.in_features 
# dropout_rate = 0.5  # You can adjust this value
# dropout_layer = nn.Dropout(p=dropout_rate)
# # Modify the fully connected layer to include the dropout layer
# last_layer_with_dropout = nn.Sequential(
#     dropout_layer,
#     nn.Linear(n_inputs, len(classes))
# )
# # Replace the original fully connected layer with the modified one
# modelZ.fc = last_layer_with_dropout
# modelZ.load_state_dict(torch.load('FineTuning10%Only.pt'))

<All keys matched successfully>

In [8]:
import numpy as np
import time

def test(loaders, model1, criterion, use_cuda):
    # monitor test loss and accuracy
    test_loss = 0.0
    correct = 0.0
    total = 0.0

    # monitor per-class accuracy
    class_correct = np.zeros(len(loaders['test'].dataset.classes))
    class_total = np.zeros(len(loaders['test'].dataset.classes))

    model1.eval()  # set model into evaluation/testing mode
    start_time = time.time()
    # Iterating over test data
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()

        model1 = model1.to(data.device)

        # forward pass: compute predicted outputs by passing inputs to the model
        output = model1(data)

        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]

        # calculate the loss
        loss = criterion(output, target)

        # update average test loss
        test_loss += ((1 / (batch_idx + 1)) * (loss.data - test_loss))

        # compare predictions to ground truth
        correct += pred.eq(target.data.view_as(pred)).sum().item()
        total += data.size(0)

        # update per-class counts
        for i in range(len(loaders['test'].dataset.classes)):
            class_correct[i] += pred[target == i].eq(target[target == i].data.view_as(pred[target == i])).sum().item()
            class_total[i] += (target == i).sum().item()

    # Print test loss, total accuracy, and accuracy for each class
    print('Test Loss: {:.6f}\n'.format(test_loss))
    print('Overall Test Accuracy: {:.3f}% ({}/{})'.format(100. * correct / total, correct, total))

    for i, class_name in enumerate(loaders['test'].dataset.classes):
        class_acc = 100. * class_correct[i] / class_total[i] if class_total[i] != 0 else 0
        print('Accuracy for class {}: {:.3f}% ({}/{})'.format(class_name, class_acc, int(class_correct[i]), int(class_total[i])))

    execution_time = time.time() - start_time
    print(f'Execution Time: {execution_time} seconds')

print('GAN Testing Result : ') 
test(loaders_transfer1, model5, criterion_transfer, use_cuda)
print('\n GLIDE Testing Result : ') 
test(loaders_transfer2, model5, criterion_transfer, use_cuda)
print('\n VQDM Testing Result : ') 
test(loaders_transfer3, model5, criterion_transfer, use_cuda)
print('\n SDM Testing Result : ') 
test(loaders_transfer4, model5, criterion_transfer, use_cuda)
print('\n Midjourney Testing Result : ') 
test(loaders_transfer5, model5, criterion_transfer, use_cuda)

# print('GAN Testing Result : ') 
# test(loaders_transfer1, dynamic_ensemble_model, criterion_transfer, use_cuda)
# print('\nGLIDE Testing Result : ') 
# test(loaders_transfer2, dynamic_ensemble_model, criterion_transfer, use_cuda)
# print('\nVQDM Testing Result : ') 
# test(loaders_transfer3, dynamic_ensemble_model, criterion_transfer, use_cuda)
# print('\nSDM Testing Result : ') 
# test(loaders_transfer4, dynamic_ensemble_model, criterion_transfer, use_cuda)

GAN Testing Result : 
Test Loss: 0.698414

Overall Test Accuracy: 48.942% (5873.0/12000.0)
Accuracy for class ai: 22.633% (1358/6000)
Accuracy for class nature: 75.250% (4515/6000)
Execution Time: 16.65420699119568 seconds

 GLIDE Testing Result : 
Test Loss: 0.706561

Overall Test Accuracy: 48.892% (5867.0/12000.0)
Accuracy for class ai: 21.850% (1311/6000)
Accuracy for class nature: 75.933% (4556/6000)
Execution Time: 17.53581953048706 seconds

 VQDM Testing Result : 
Test Loss: 0.679758

Overall Test Accuracy: 56.067% (6728.0/12000.0)
Accuracy for class ai: 35.950% (2157/6000)
Accuracy for class nature: 76.183% (4571/6000)
Execution Time: 18.392539739608765 seconds

 SDM Testing Result : 
Test Loss: 0.739727

Overall Test Accuracy: 49.825% (9965.0/20000.0)
Accuracy for class ai: 0.160% (16/10000)
Accuracy for class nature: 99.490% (9949/10000)
Execution Time: 26.437470197677612 seconds

 Midjourney Testing Result : 
Test Loss: 0.709753

Overall Test Accuracy: 48.658% (5839.0/12000.0

In [4]:
dir_GAN = 'Dataset/GAN/BigGan/val'
dir_GLIDE = 'Dataset/GLIDE/val'
dir_VQDM = 'Dataset/VQDM/val'
dir_SDM = 'Dataset/SD/test'
dir_Midjourney = 'Dataset/MidjourneyZip/Midjourney/val'


test_GAN = datasets.ImageFolder(dir_GAN, transform=data_transforms["test_transforms"])
test_GLIDE = datasets.ImageFolder(dir_GLIDE, transform=data_transforms["test_transforms"])
test_VQDM = datasets.ImageFolder(dir_VQDM, transform=data_transforms["test_transforms"])
test_SDM = datasets.ImageFolder(dir_SDM, transform=data_transforms["test_transforms"])
test_Midjourney = datasets.ImageFolder(dir_Midjourney, transform=data_transforms["test_transforms"])

print('GAN Testing:')
GAN_count = len(test_GAN)
GAN_idx = indices[:GAN_count]
print("Test", GAN_count)
test_sampler = SubsetRandomSampler(GAN_idx)
testloaderGAN = torch.utils.data.DataLoader(test_GAN, batch_size = 16, sampler = test_sampler,num_workers=4)
loaders_transfer1 = {'test': testloaderGAN}   

print('GLIDE Testing:')
GLIDE_count = len(test_GLIDE)
GLIDE_idx = indices[:GLIDE_count]
print("Test", GLIDE_count)
test_sampler = SubsetRandomSampler(GLIDE_idx)
testloaderGLIDE = torch.utils.data.DataLoader(test_GLIDE, batch_size = 16, sampler = test_sampler,num_workers=4)
loaders_transfer2 = {'test': testloaderGLIDE}  

print('VQDM Testing:')
VQDM_count = len(test_VQDM)
VQDM_idx = indices[:VQDM_count]
print("Test", VQDM_count)
test_sampler = SubsetRandomSampler(VQDM_idx)
testloaderVQDM = torch.utils.data.DataLoader(test_VQDM, batch_size = 16, sampler = test_sampler,num_workers=4)
loaders_transfer3 = {'test': testloaderVQDM}  

print('SDM Testing:')
SDM_count = len(test_SDM)
SDM_idx = indices[:SDM_count]
print("Test", SDM_count)
test_sampler = SubsetRandomSampler(SDM_idx)
testloaderSDM = torch.utils.data.DataLoader(test_SDM, batch_size = 16, sampler = test_sampler,num_workers=4)
loaders_transfer4 = {'test': testloaderSDM}  

print('Midjourney Testing:')
Midjourney_count = len(test_Midjourney)
Midjourney_idx = indices[:Midjourney_count]
print("Test", Midjourney_count)
test_sampler = SubsetRandomSampler(Midjourney_idx)
testloaderMidjourney = torch.utils.data.DataLoader(test_Midjourney, batch_size = 16, sampler = test_sampler,num_workers=4)
loaders_transfer5 = {'test': testloaderMidjourney}  



GAN Testing:
Test 12000
GLIDE Testing:
Test 12000
VQDM Testing:
Test 12000
SDM Testing:
Test 20000
Midjourney Testing:
Test 12000


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

# Set models to evaluation mode
model1.eval()
model2.eval()
model3.eval()
model4.eval()

# Initialize weights equally
w1, w2, w3, w4 = 0.25, 0.25, 0.25, 0.25

# Define a new class for the ensemble
class DynamicWeightedEnsemble(nn.Module):
    def __init__(self, model1, model2, model3, model4):
        super(DynamicWeightedEnsemble, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model4 = model4

        # Convert weights to parameters so they can be updated during optimization
        self.weights = nn.Parameter(torch.tensor([w1, w2, w3, w4], requires_grad=True))

    def forward(self, x):
        out1 = self.model1(x)
        out2 = self.model2(x)
        out3 = self.model3(x)
        out4 = self.model4(x)

        # Combine the outputs using dynamic weights
        weights_softmax = F.softmax(self.weights, dim=0)
        # print("Softmax Weights:", weights_softmax)
        # out = weights_softmax[0] * out1 + weights_softmax[1] * out2 + weights_softmax[2] * out3 + weights_softmax[3] * out4
        out = 0.3180 * out1 + 0.1681 * out2 + 0.1695 * out3 + 0.3445 * out4
        return out

# Create an instance of the dynamic weighted ensemble model
dynamic_ensemble_model = DynamicWeightedEnsemble(model1, model2, model3, model4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dynamic_ensemble_model.to(device)
dynamic_ensemble_model.load_state_dict(torch.load('Model/GATING.pt'))
# dynamic_ensemble_model.load_state_dict(torch.load('Model/GATING.pt'))



<All keys matched successfully>